<a href="https://colab.research.google.com/github/PranitUkhalkar/FUTURE_ML_01/blob/main/Sales_Forecasting_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print(sys.version)

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [2]:
!pip install prophet


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet

print("colab environment ready for Sales Forcasting")

colab environment ready for Sales Forcasting
